#                     Projeto Final CDD- Prever a nota de um filme no IDB

In [237]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import seaborn as sns
import numpy as np
from scipy.stats import norm, probplot
import statsmodels.api as sm
from mpl_toolkits.mplot3d import Axes3D
import json

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)

In [238]:
### Função utilitária para fazer a regressão com constante adicionada
def regress(X,Y):
    X_cp = sm.add_constant(X)
    model = sm.OLS(Y,X_cp)
    results = model.fit()
    return results

In [239]:
dados_filmes= pd.read_csv("tmdb_5000_movies.csv")

In [240]:
dados_filmes.drop(dados_filmes[dados_filmes["budget"]==0].index,inplace=True)

# Começaremos analisando os dados do Dataset

## As variáveis numéricas que analisaremos, para ver se há uma relação com a nota do filme, será as numéricas. Estas serão:

### title, runtime budget, revenue, popularity, vote_average, vote_count

## Construindo o Dataframe

In [241]:
dados_novos = dados_filmes[["title","runtime","budget","revenue", "vote_average", "vote_count", "popularity","genres"]].copy()

#### Criação de colunas com o nome dos generos, sendo 1 para quando o filme é daquela categoria e 0 para quando não é , isso foi feito para tirar a coluna genre

In [242]:
genres = set()
for x in dados_novos['genres']:
    for g in json.loads(x):
        genres.add(g["name"])

genres

{'Action',
 'Adventure',
 'Animation',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Foreign',
 'History',
 'Horror',
 'Music',
 'Mystery',
 'Romance',
 'Science Fiction',
 'TV Movie',
 'Thriller',
 'War',
 'Western'}

#### Criacão do novo dataframe incluindo as novas colunas de gênero

In [243]:
def extrai_generos(item):
    generos = {'c_' + x: 0 for x in ['Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 
                              'Documentary', 'Drama', 'Family', 'Fantasy', 'Foreign', 
                              'History', 'Horror', 'Music', 'Mystery', 'Romance', 
                              'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western']}
    for g in json.loads(item):
        generos['c_' + g['name']] += 1
    return generos

dados_novos_teste = pd.DataFrame(list(dados_novos['genres'].apply(extrai_generos).values))

dados_novos_junto = dados_novos.join(dados_novos_teste)

dados_novos_junto.set_index("title",inplace=True)

In [244]:
dados_novos_junto.drop(columns= "genres", inplace=True)



## Variáveis para a análise

**Dicionário de dados:**


Variável | Descrição
:---:|---:
title | Título do filme
runtime | Tempo de filme
budget | Orçamento
revenue | Receita
vote_average | Média dos votos
vote_count | Número de votos das pessoas
popularity | Popularidade
genres | Os diferentes gêneros de filme

## Com a ajuda do comando Describe, poderemos analisar fatores importantes do Dataframe


 

In [245]:
dados_novos_junto.describe()

,runtime,budget,revenue,vote_average,vote_count,popularity,c_Action,c_Adventure,c_Animation,c_Comedy,...,c_History,c_Horror,c_Music,c_Mystery,c_Romance,c_Science Fiction,c_TV Movie,c_Thriller,c_War,c_Western
count,3764.000000,3.766000e+03,3.766000e+03,3766.000000,3766.000000,3766.000000,3221.000000,3221.000000,3221.000000,3221.000000,...,3221.000000,3221.000000,3221.000000,3221.000000,3221.000000,3221.000000,3221.000000,3221.000000,3221.000000,3221.000000
mean,109.324389,3.704284e+07,1.039547e+08,6.226474,856.496814,25.984641,0.282521,0.208010,0.060540,0.350512,...,0.044707,0.105247,0.034772,0.074201,0.174170,0.127600,0.000931,0.289662,0.035082,0.017075
std,20.753408,4.264651e+07,1.776399e+08,0.966228,1344.785690,34.438253,0.450295,0.405947,0.238522,0.477204,...,0.206691,0.306919,0.183230,0.262138,0.379314,0.333696,0.030509,0.453676,0.184016,0.129573
min,0.000000,1.000000e+00,0.000000e+00,0.000000,0.000000,0.001586,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,95.000000,8.000000e+06,6.009736e+06,5.700000,114.000000,7.974990,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,105.500000,2.300000e+07,3.883137e+07,6.300000,365.500000,17.604955,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,120.000000,5.000000e+07,1.220873e+08,6.900000,969.250000,33.925801,1.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,338.000000,3.800000e+08,2.787965e+09,10.000000,13752.000000,875.581305,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### Verificando valores nulos das variáveis em certa coluna

In [246]:
np.sum(df.isnull())

runtime              0
budget               0
revenue              0
vote_average         0
vote_count           0
popularity           0
c_Action             0
c_Adventure          0
c_Animation          0
c_Comedy             0
c_Crime              0
c_Documentary        0
c_Drama              0
c_Family             0
c_Fantasy            0
c_Foreign            0
c_History            0
c_Horror             0
c_Music              0
c_Mystery            0
c_Romance            0
c_Science Fiction    0
c_TV Movie           0
c_Thriller           0
c_War                0
c_Western            0
dtype: int64

### Tirando os dados Nan

In [247]:
df = dados_novos_junto

df = df.dropna()

## Separação das  variáveis

In [248]:
Y = df[ "vote_average"]

In [249]:
X = df[["runtime","budget","revenue","vote_count", "popularity","c_Action", "c_Adventure","c_Animation","c_Comedy","c_History",
        "c_Horror","c_Music","c_Mystery","c_Romance","c_Science Fiction","c_TV Movie","c_Thriller","c_War","c_Western"]]

## Uso da função Regress

In [250]:
results = regress(X,Y)
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           vote_average   R-squared:                       0.363
Model:                            OLS   Adj. R-squared:                  0.359
Method:                 Least Squares   F-statistic:                     95.94
Date:                Thu, 08 Nov 2018   Prob (F-statistic):          1.42e-295
Time:                        15:04:26   Log-Likelihood:                -3389.3
No. Observations:                3220   AIC:                             6819.
Df Residuals:                    3200   BIC:                             6940.
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 4.4779      0.073     61.722      0.000       4.336       4.620
runtime               0.0160      0.001     25.557      0.000       0.015       0.017
budget            -6.789e-09   4.26e-10    -15.941      0.000   -7.62e-09   -5.95e-09
revenue            8.025e-11    1.2e-10      0.670      0.503   -1.55e-10    3.15e-10
vote_count            0.0003   1.63e-05     16.980      0.000       0.000       0.000
popularity            0.0011      0.001      2.082      0.037    6.29e-05       0.002
c_Action             -0.0096      0.033     -0.291      0.771      -0.075       0.055
c_Adventure          -0.0911      0.035     -2.568      0.010      -0.161      -0.022
c_Animation           0.2458      0.056      4.427      0.000       0.137       0.355
c_Comedy             -0.0115      0.029     -0.393      0.694      -0.069       0.046
c_History            -0.1066      0.065     -1.644      0.100      -0.234       0.021
c_Horror              0.0497      0.043      1.165      0.244      -0.034       0.133
c_Music              -0.0689      0.068     -1.011      0.312      -0.202       0.065
c_Mystery            -0.0551      0.049     -1.114      0.265      -0.152       0.042
c_Romance            -0.0167      0.035     -0.480      0.631      -0.085       0.051
c_Science Fiction    -0.0022      0.039     -0.057      0.955      -0.079       0.075
c_TV Movie            0.3102      0.403      0.771      0.441      -0.479       1.099
c_Thriller           -0.0252      0.032     -0.777      0.437      -0.089       0.038
c_War                -0.0653      0.072     -0.909      0.364      -0.206       0.076
c_Western             0.0056      0.096      0.059      0.953      -0.183       0.195
==============================================================================
Omnibus:                      263.254   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              437.514
Skew:                          -0.604   Prob(JB):                     9.89e-96
Kurtosis:                       4.342   Cond. No.                     7.47e+09
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.47e+09. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Adotamos um ALFA de 5%, então tiramos os valores que tem um P>T maior que o ALFA 

In [251]:
X = df[["runtime","budget","revenue","vote_count", "popularity"]]

Separando o dataframe em teste e treinamento

In [252]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=RANDOM_SEED)

Treinando, criando , testando e calculando o modelo de regressão linear

In [253]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

model = LinearRegression()       #cria
model.fit(X_train, y_train)        #treina 
y_pred = model.predict(X_test)      #testa 

1 - mean_squared_error(y_test, y_pred) / np.var(y_test)

0.38094718882784806

In [254]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

model = RandomForestRegressor()       #cria
model.fit(X_train, y_train)        #treina 
y_pred = model.predict(X_test)      #testa 

1 - mean_squared_error(y_test, y_pred) / np.var(y_test)

0.4281948562992448